# Perform single-cell level quality control

In [1]:
import os
import pathlib
import sys

import pandas as pd
from arg_parsing_utils import parse_args
from cosmicqc import find_outliers
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot/NF1_organoid_data")).resolve(),
    root_dir,
)

In [2]:
if not in_notebook:
    args = parse_args()
    image_based_profiles_subparent_name = args["image_based_profiles_subparent_name"]

else:
    image_based_profiles_subparent_name = "image_based_profiles"

## Load in each single-cell level profile per patient and process

1. Load in the single-cell data (add `patient_id` column).
2. Load in respective organoid qc data (only metadata and cqc columns) to already flag cells that come from a flagged organoid.
   - Also add a flag for if single-cells do not have an organoid segmentation (`parent_organoid` == -1).
   - Also add flag for if the `object_id` for a single-cell is NaN.
3. Concat single-cell data together.

In [ ]:
# Path to patient folders
path_to_patients = pathlib.Path(f"{profile_base_dir}/data/")

dfs = []
for patient_folder in path_to_patients.iterdir():
    single_cell_file = (
        patient_folder
        / f"{image_based_profiles_subparent_name}/2.annotated_profiles"
        / "sc_anno.parquet"
    )
    organoid_flags_file = (
        patient_folder
        / f"{image_based_profiles_subparent_name}/3.qc_profiles"
        / "organoid_flagged_outliers.parquet"
    )

    if single_cell_file.exists():
        sc_df = pd.read_parquet(single_cell_file)
        sc_df["Metadata_patient_id"] = patient_folder.name

        # Default QC flags
        sc_df["Metadata_cqc.organoid_flagged"] = False
        sc_df["Metadata_cqc.nan_detected"] = (
            sc_df[["Metadata_object_id", "Area.Size.Shape_Nuclei_VOLUME"]]
            .isna()
            .any(axis=1)
        )
        sc_df["Metadata_cqc.missing_parent_organoid"] = (
            sc_df["Metadata_parent_organoid"] == -1
        )

        if organoid_flags_file.exists():
            organoid_flags_df = pd.read_parquet(organoid_flags_file)[
                ["Metadata_object_id", "Metadata_image_set"]
                + [
                    col
                    for col in pd.read_parquet(organoid_flags_file).columns
                    if col.startswith("cqc")
                ]
            ]

            # Get flagged (object_id, image_set) pairs
            flagged_pairs = set(
                organoid_flags_df.loc[
                    organoid_flags_df.filter(like="cqc").any(axis=1),
                    ["Metadata_object_id", "Metadata_image_set"],
                ].itertuples(index=False, name=None)
            )

            # Flag SC rows where both parent_organoid & image_set match a flagged organoid
            sc_df["Metadata_cqc.organoid_flagged"] = sc_df.apply(
                lambda row: (row["Metadata_parent_organoid"], row["Metadata_image_set"])
                in flagged_pairs,
                axis=1,
            )

        dfs.append(sc_df)

orig_single_cell_profiles_df = pd.concat(dfs, ignore_index=True)

print(orig_single_cell_profiles_df.shape)
orig_single_cell_profiles_df.head()

(17273, 5984)


,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,...,Texture_Cytoplasm_Mito_Variance_256.3,Metadata_Target,Metadata_Class,Metadata_Therapeutic_Categories,Metadata_patient,Metadata_tumor,Metadata_patient_id,cqc.organoid_flagged,cqc.nan_detected,cqc.missing_parent_organoid
0,NF0014_T2,135,nM,10.0,Staurosporine,C11-4,C11,-1,383105.0,888.438049,...,36.711999,Apoptosis induction,Small Molecule,Experimental,NF0014,T2,NF0014_T2,False,False,True
1,NF0014_T2,240,nM,10.0,Staurosporine,C11-4,C11,-1,59199.0,819.387146,...,1.146901,Apoptosis induction,Small Molecule,Experimental,NF0014,T2,NF0014_T2,False,False,True
2,NF0014_T2,38,uM,1.0,Selumetinib,D11-5,D11,-1,102593.0,732.440735,...,3.012070,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor,NF0014,T2,NF0014_T2,False,False,True
3,NF0014_T2,90,uM,1.0,Selumetinib,D11-5,D11,-1,215890.0,794.163879,...,4.351801,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor,NF0014,T2,NF0014_T2,False,False,True
4,NF0014_T2,138,uM,1.0,Selumetinib,D11-5,D11,-1,40802.0,642.178467,...,5.160550,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor,NF0014,T2,NF0014_T2,False,False,True


## Detect outlier single-cells using the non-flagged data

We will attempt to detect instances of poor quality segmentations using the nuclei compartment as the base. The conditions we are using are as follows:

1. Abnormally small or large nuclei using `Volume`
2. Abnormally high `mass displacement` in the nuclei for instances of mis-segmentation of background/no longer in-focus

In [ ]:
# Set the metadata columns to be used in the QC process
metadata_columns = [
    "Metadata_patient_id",
    "Metadata_image_set",
    "Metadata_object_id",
    "Metadata_parent_organoid",
    "Area.Size.Shape_Nuclei_CENTER.X",
    "Area.Size.Shape_Nuclei_CENTER.Y",
    "Metadata_cqc.nan_detected",
    "Metadata_cqc.organoid_flagged",
    "Metadata_cqc.missing_parent_organoid",
]

In [ ]:
# Process each plate (patient_id) independently in the combined dataframe
for plate_name, plate_df in orig_single_cell_profiles_df.groupby("Metadata_patient_id"):
    print(f"Processing plate: {plate_name}")

    # Make a contiguous copy to prevent DataFrame fragmentation
    plate_df = plate_df.copy()

    # Only process the rows that are not flagged
    filtered_plate_df = plate_df[
        ~(
            plate_df["Metadata_cqc.nan_detected"]
            | plate_df["Metadata_cqc.organoid_flagged"]
            | plate_df["Metadata_cqc.missing_parent_organoid"]
        )
    ]

    # --- Find size based nuclei outliers ---
    print("Finding small nuclei outliers...")
    small_nuclei_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Area.Size.Shape_Nuclei_VOLUME": -1,  # Detect very small nuclei
        },
    )

    # Ensure the column exists before assignment
    plate_df["Metadata_cqc.small_nuclei_outlier"] = False
    plate_df.loc[small_nuclei_outliers.index, "Metadata_cqc.small_nuclei_outlier"] = (
        True
    )

    print("Finding large nuclei outliers...")
    large_nuclei_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Area.Size.Shape_Nuclei_VOLUME": 2,  # Detect very large nuclei
        },
    )

    # Ensure the column exists before assignment
    plate_df["Metadata_cqc.large_nuclei_outlier"] = False
    plate_df.loc[large_nuclei_outliers.index, "Metadata_cqc.large_nuclei_outlier"] = (
        True
    )

    # --- Find mass displacement based nuclei outliers ---
    print("Finding high mass displacement outliers...")
    high_mass_displacement_outliers = find_outliers(
        df=filtered_plate_df,
        metadata_columns=metadata_columns,
        feature_thresholds={
            "Intensity_Nuclei_DNA_MASS.DISPLACEMENT": 2,  # Detect high mass displacement
        },
    )

    # Ensure the column exists before assignment
    plate_df["Metadata_cqc.mass_displacement_outlier"] = False
    plate_df.loc[
        high_mass_displacement_outliers.index, "Metadata_cqc.mass_displacement_outlier"
    ] = True

    # Print number of outliers (only in filtered rows)
    small_count = filtered_plate_df.index.intersection(
        small_nuclei_outliers.index
    ).shape[0]
    large_count = filtered_plate_df.index.intersection(
        large_nuclei_outliers.index
    ).shape[0]
    high_mass_count = filtered_plate_df.index.intersection(
        high_mass_displacement_outliers.index
    ).shape[0]

    print(f"Small nuclei outliers found: {small_count}")
    print(f"Large nuclei outliers found: {large_count}")
    print(f"High mass displacement outliers found: {high_mass_count}")

    # Save updated plate_df with flag columns included
    output_folder = path_to_patients / plate_name / "image_based_profiles/3.qc_profiles"
    output_folder.mkdir(parents=True, exist_ok=True)
    output_file = output_folder / "sc_flagged_outliers.parquet"
    plate_df.to_parquet(output_file, index=False)
    print(f"Saved single-cell profiles with outlier flags to {output_file}\n")

Processing plate: NF0014_T1
Finding small nuclei outliers...
Number of outliers: 183 (17.48%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: 342.0
Area.Size.Shape_Nuclei_VOLUME Max: 26930.0
Finding large nuclei outliers...
Number of outliers: 48 (4.58%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: 177624.0
Area.Size.Shape_Nuclei_VOLUME Max: 421250.0
Finding high mass displacement outliers...
Number of outliers: 21 (2.01%)
Outliers Range:
Intensity_Nuclei_DNA_MASS.DISPLACEMENT Min: 1545.5691
Intensity_Nuclei_DNA_MASS.DISPLACEMENT Max: 1676.4082
Small nuclei outliers found: 183
Large nuclei outliers found: 48
High mass displacement outliers found: 21
Saved single-cell profiles with outlier flags to /home/lippincm/mnt/bandicoot/NF1_organoid_data/data/NF0014_T1/image_based_profiles/1a.qc_profiles/sc_flagged_outliers.parquet

Processing plate: NF0014_T2
Finding small nuclei outliers...
Number of outliers: 0 (0.00%)
Outliers Range:
Area.Size.Shape_Nuclei_VOLUME Min: nan
Area.Size.S

In [12]:
# Print example output of the flagged single-cell profiles
print(f"Example flagged single-cell profiles: {plate_name}")
print(plate_df.shape)
plate_df.head()

Example flagged single-cell profiles: NF0030_T1
(2787, 5987)


,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,...,Metadata_Therapeutic_Categories,Metadata_patient,Metadata_tumor,Metadata_patient_id,cqc.organoid_flagged,cqc.nan_detected,cqc.missing_parent_organoid,cqc.small_nuclei_outlier,cqc.large_nuclei_outlier,cqc.mass_displacement_outlier
3649,NF0030_T1,4,nM,10.0,Staurosporine,C11-4,C11,-1,23908.0,663.728821,...,Experimental,NF0030,T1,NF0030_T1,False,False,True,False,False,False
3650,NF0030_T1,21,nM,10.0,Staurosporine,C11-4,C11,-1,31793.0,764.801697,...,Experimental,NF0030,T1,NF0030_T1,False,False,True,False,False,False
3651,NF0030_T1,21,nM,10.0,Staurosporine,C11-4,C11,-1,31793.0,764.801697,...,Experimental,NF0030,T1,NF0030_T1,False,False,True,False,False,False
3652,NF0030_T1,55,nM,10.0,Staurosporine,C11-4,C11,-1,33245.0,631.298828,...,Experimental,NF0030,T1,NF0030_T1,False,False,True,False,False,False
3653,NF0030_T1,55,nM,10.0,Staurosporine,C11-4,C11,-1,33245.0,631.298828,...,Experimental,NF0030,T1,NF0030_T1,False,False,True,False,False,False
